# Auto-generating the Cosmology Meetings Event List

_Phil Marshall, August 2018_

We have a Google calendar with an `ical` address: this notebook scrapes the events out of that calendar and produces a restructured text table of them.

## Requirements

We'll need tooling to interpret the `ical` file - let's use [`pyICSParser`](https://pypi.org/project/pyICSParser/) by [@oberron](https://github.com/oberron/annum) - and we'll use `pandas` to manipulate the resulting table.

In [ ]:
# ! pip install --upgrade pyICSParser

In [ ]:
# ! pip install --upgrade pandas

## Scraping the Calendar

This is a straightforward `wget` call.

In [ ]:
# Calendar ical address:
url = "https://calendar.google.com/calendar/ical/a6aumk8bjbhhb9aaa6hg60f19c%40group.calendar.google.com/public/basic.ics"

In [ ]:
icsfile = "EVENTS.ics"
! wget $url -O $icsfile

## Parsing the Calendar

Here's where we use `pyICSParser` to read the `ical` file, and present the information in a nice table.

In [ ]:
from pyICSParser import pyiCalendar as ical

mycal = ical.iCalendar()
#start and end are string objects of yyyymmdd type
mycal.local_load(icsfile)

In [ ]:
mycal.parse_loaded()

In [ ]:
# help(mycal)

In [ ]:
len(mycal.events)

Here's what one of these event items looks like:

In [ ]:
mycal.events[12]

Let's load the event information into a pandas dataframe so we can work with it more easily.

In [ ]:
import pandas
df = pandas.DataFrame(columns=['Meeting','Start Date','End Date','URL'])

k=0
for event in mycal.events:
    # Pull out the URL alone, without the html tags:
    strings = event['DESCRIPTION']['val'].split('"')
    if len(strings) == 3:
        url = strings[1]
    else:
        url = ''
    # Add a row to the data frame:
    df.loc[k] = pandas.Series({'Meeting':event['SUMMARY']['val'], 
                               'Start Date':event['DTSTART']['val'].isoformat(), 
                               'End Date':event['DTEND']['val'].isoformat(),
                               'URL':url 
                              })
    k += 1

We need to sort this table by start date, and only keep the events in the future.

In [ ]:
import datetime
today = datetime.date.today().isoformat()
print today

In [ ]:
sorted_df = df.sort_values(by="Start Date")
final_df = sorted_df[sorted_df['Start Date'] >= today]
final_df

## Writing the Event List

We want a simple markdown table, exported to a file, where the name of the meeting is a hyperlink, if the URL is available.

In [ ]:
def easy_pandas_df_to_markdown_table(df):
    fmt = ['---' for i in range(len(df.columns))]
    df_fmt = pandas.DataFrame([fmt], columns=df.columns)
    df_formatted = pandas.concat([df_fmt, df])
    df_formatted.to_csv("EVENTS.md", sep="|", index=False)
    return

def row_by_row_pandas_df_to_markdown_table(df):
    f = open("EVENTS.md", "w")
    f.write("Meeting | Start Date | End Date\n")    
    f.write("--------|------------|---------\n")
    for index, row in df.iterrows():
        if row['URL'] == '':
            f.write(row['Meeting']+" | "+row['Start Date']+" | "+row['End Date']+"\n")
        else:
            f.write("["+row['Meeting']+"]("+row['URL']+") | "+row['Start Date']+" | "+row['End Date']+"\n")
    f.close()
    return

# Call the function with our filtered, sorted data frame:
row_by_row_pandas_df_to_markdown_table(final_df)

In [ ]:
! wc -l EVENTS.md

In [ ]:
! cat EVENTS.md

Done! This file can now be deployed to the `CosmologyMeetings` website.